# Training LLMs: Complete Pipeline from Pretraining to RLHF

This notebook demonstrates the full training pipeline for large language models through three main stages:

1. **Pretraining** - Training a model from scratch on raw text
2. **Supervised Fine-tuning (SFT)** - Teaching instruction following  
3. **Reinforcement Learning from Human Feedback (RLHF)** - Aligning with human preferences

We'll build a small Mistral model and train it end-to-end to understand each stage.

# Stage 1: Pretraining

In this section, we'll train a model from scratch using next-token prediction on raw Wikipedia text. This teaches the model basic language understanding and generation capabilities.

## Data Loading

Here we're downloading a small subset (1000 samples) of the Wikipedia dataset for demonstration. This gives us raw text data that we'll use for pretraining our model from scratch.

In [3]:
from datasets import load_dataset

wiki_data = load_dataset(
    "wikimedia/wikipedia",   
    "20231101.en", 
    split="train[:1000]"
)

Resolving data files:   0%|          | 0/41 [00:00<?, ?it/s]

In [2]:
print(wiki_data['text'][0][:1000])

Anarchism is a political philosophy and movement that is skeptical of all justifications for authority and seeks to abolish the institutions it claims maintain unnecessary coercion and hierarchy, typically including nation-states, and capitalism. Anarchism advocates for the replacement of the state with stateless societies and voluntary free associations. As a historically left-wing movement, this reading of anarchism is placed on the farthest left of the political spectrum, usually described as the libertarian wing of the socialist movement (libertarian socialism).

Humans have lived in societies without formal hierarchies long before the establishment of states, realms, or empires. With the rise of organised hierarchical bodies, scepticism toward authority also rose. Although traces of anarchist ideas are found all throughout history, modern anarchism emerged from the Enlightenment. During the latter half of the 19th and the first decades of the 20th century, the anarchist movement f

Let's split the data into train and test

In [3]:
wiki_data = wiki_data.train_test_split(test_size=0.2)

In [4]:
wiki_data

DatasetDict({
    train: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 800
    })
    test: Dataset({
        features: ['id', 'url', 'title', 'text'],
        num_rows: 200
    })
})

## Tokenization Process

This shows what tokenization looks like - converting raw text into numerical tokens that the model can process.

I am going to train a model from scratch I am going to use the Mistral architecture as base. I will use the same tokenize to move from text data to the input index data. I had to run `huggingface-cli login` and enter my token to download this specific model

In [5]:
from transformers import AutoTokenizer

base_model_id = 'mistralai/Mistral-7B-v0.1'
tokenizer = AutoTokenizer.from_pretrained(base_model_id)
tokenizer.pad_token = tokenizer.eos_token

The padding token was missing

In [6]:
tokenizer.special_tokens_map

{'bos_token': '<s>',
 'eos_token': '</s>',
 'unk_token': '<unk>',
 'pad_token': '</s>'}

Tokenizing the data is easy

In [7]:
outputs = tokenizer(
    wiki_data['train']['text'][0:10],
)

This function processes our text data in batches:
- `truncation=True, max_length=512`: Cuts off text longer than 512 tokens
- `padding='max_length'`: Pads shorter sequences to 512 tokens with pad tokens
- `return_tensors="pt"`: Returns PyTorch tensors instead of lists
- `remove_columns`: Removes original text columns, keeping only tokenized data

In [8]:
max_length = 512

def tokenize_function(examples):
    return tokenizer(
        examples['text'], 
        truncation=True, 
        max_length=max_length, 
        padding='max_length', # longuest 
        return_tensors="pt", 
        add_special_tokens=True
    )

tokenized_datasets = wiki_data.map(
    tokenize_function, 
    batched=True, 
    remove_columns=['id', 'url', 'title', 'text']
)

Map:   0%|          | 0/800 [00:00<?, ? examples/s]

Map:   0%|          | 0/200 [00:00<?, ? examples/s]

In [9]:
tokenizer.padding_side

'left'

In [10]:
tokenizer.pad_token_id

2

Notice that `padding_side='left'` means padding tokens are added to the beginning of sequences, and `pad_token_id=2` shows padding uses the same token as EOS (end of sequence).

## Model Architecture

First we load the default Mistral configuration to see the full-size model parameters. This shows a 7B parameter model with 32 layers, 4096 hidden dimensions, etc.

In [11]:
from transformers import MistralForCausalLM, MistralConfig
config = MistralConfig()

In [12]:
config

MistralConfig {
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "head_dim": null,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 131072,
  "model_type": "mistral",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "rms_norm_eps": 1e-06,
  "rope_theta": 10000.0,
  "sliding_window": 4096,
  "tie_word_embeddings": false,
  "transformers_version": "4.52.4",
  "use_cache": true,
  "vocab_size": 32000
}

Here we can see the default Mistral configuration showing a 7B parameter model with 32 layers, 4096 hidden dimensions, etc. This is too large for our demonstration.

Now we create a much smaller model configuration for demonstration:
- `hidden_size=768`: Reduced from 4096 to make training faster
- `num_hidden_layers=4`: Only 4 layers instead of 32
- `num_attention_heads=16`: Reduced attention heads
- `intermediate_size=3072`: Smaller feed-forward network
- `max_position_embeddings=512`: Matches our sequence length

In [13]:
config = MistralConfig(
    hidden_size=768,
    sliding_window=768,
    intermediate_size=3072,
    max_position_embeddings=max_length,
    num_attention_heads=16,  
    num_hidden_layers=4,
)

This creates our small Mistral model with the reduced configuration. The model structure shows 4 decoder layers with our specified dimensions.

In [17]:
model = MistralForCausalLM(config)
model.config.pad_token_id = tokenizer.pad_token_id

In [18]:
model

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 768)
    (layers): ModuleList(
      (0-3): 4 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (k_proj): Linear(in_features=768, out_features=384, bias=False)
          (v_proj): Linear(in_features=768, out_features=384, bias=False)
          (o_proj): Linear(in_features=768, out_features=768, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=768, out_features=3072, bias=False)
          (up_proj): Linear(in_features=768, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=768, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((768,), eps=1e-06)
        (post_attention_layernorm): MistralRMSNorm((768,), eps=1e-06)
      )
    )
    (norm): MistralRMSNorm((768,), eps=1e-06)
    (rotary_emb):

Our small model has ~84M parameters compared to the 7B in the original Mistral model. This makes it feasible to train on modest hardware.

In [19]:
model_size = sum(t.numel() for t in model.parameters())
model_size

84548352

## Data Collation for Language Modeling

`DataCollatorForLanguageModeling` with `mlm=False` sets up causal language modeling (predicting next tokens). It automatically creates labels by shifting input_ids by one position.

In [21]:
from transformers import DataCollatorForLanguageModeling

data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)


This shows what a batch looks like after collation. We have 10 sequences, each tokenized and ready for training. The data collator creates batches from our tokenized data. Let's see what a batch looks like with 10 sequences:

In [22]:
out = data_collator([
    tokenized_datasets["train"][i] for i in range(10)
])

This shows the first sequence's input_ids - the tokenized text ready for training. These are the numerical tokens the model will learn from.

In [24]:
out['input_ids'][0]

tensor([    1,  8786,   591,   509,   325, 28745, 10407,   753, 19840,   442,
        16320,   905,   460,  5878,   973,   442,  9893,   302, 19756, 28725,
          442,   905,   395, 14014,   643,   477,   736, 28723, 19756,   349,
         1269,   582,   302,  4118, 18433,  1218, 28725,   302,   690, 28705,
        12184,   407, 13716, 28725,   320,  1150,   849, 28713, 28725,   382,
        21364,   293,   304,   500, 15843, 23810,   460,   272,  7639, 28723,
          415,   989,  2191, 14028, 14382,   486,  8786,   591,   509,   460,
        12184,   407, 28709,   304,   384,  1900,   325,  1237, 16320, 26228,
          302,   272, 10407,   753,  3842,   557,   304,  1287,  8786,   591,
          509,   460,   287,  5708,   840,   297,  8711,   972,  9938, 12184,
          407, 28709,   304,   384,  1900, 28723,    13,    13, 11278,    13,
         1014, 21864,  4389,   302,   272,  1141, 16320,   325,  6570, 28721,
        18215, 28731,   349,   486,  1295,   377,   324,   315, 

In [25]:
out['labels'][0]

tensor([    1,  8786,   591,   509,   325, 28745, 10407,   753, 19840,   442,
        16320,   905,   460,  5878,   973,   442,  9893,   302, 19756, 28725,
          442,   905,   395, 14014,   643,   477,   736, 28723, 19756,   349,
         1269,   582,   302,  4118, 18433,  1218, 28725,   302,   690, 28705,
        12184,   407, 13716, 28725,   320,  1150,   849, 28713, 28725,   382,
        21364,   293,   304,   500, 15843, 23810,   460,   272,  7639, 28723,
          415,   989,  2191, 14028, 14382,   486,  8786,   591,   509,   460,
        12184,   407, 28709,   304,   384,  1900,   325,  1237, 16320, 26228,
          302,   272, 10407,   753,  3842,   557,   304,  1287,  8786,   591,
          509,   460,   287,  5708,   840,   297,  8711,   972,  9938, 12184,
          407, 28709,   304,   384,  1900, 28723,    13,    13, 11278,    13,
         1014, 21864,  4389,   302,   272,  1141, 16320,   325,  6570, 28721,
        18215, 28731,   349,   486,  1295,   377,   324,   315, 

## Training Configuration and Execution

Now we set up the training arguments and run the pretraining:

The training shows typical pretraining behavior: high initial loss that gradually decreases. The final loss of indicates the model is learning to predict next tokens, though it would need much more training to be truly useful.

In [26]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="mistral-pretraining",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    num_train_epochs=1,
    push_to_hub=True,
    report_to="none", 
)

trainer = Trainer(
    model=model,
    processing_class=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
)

trainer.train()

/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
500,7.235400


TrainOutput(global_step=800, training_loss=7.001226654052735, metrics={'train_runtime': 50.8679, 'train_samples_per_second': 15.727, 'train_steps_per_second': 15.727, 'total_flos': 147388052275200.0, 'train_loss': 7.001226654052735, 'epoch': 1.0})

We need to log into Hugging Face Hub to push our trained models: 
```huggingface-cli login```

In [28]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-pretraining/commit/1baa25ded579dffd9050a56daa4d814402187172', commit_message='End of training', commit_description='', oid='1baa25ded579dffd9050a56daa4d814402187172', pr_url=None, repo_url=RepoUrl('https://huggingface.co/damienbenveniste/mistral-pretraining', endpoint='https://huggingface.co', repo_type='model', repo_id='damienbenveniste/mistral-pretraining'), pr_revision=None, pr_num=None)

Let's test our pretrained model with a simple text generation pipeline. Testing our pretrained model shows it generates mostly gibberish - this is expected since it's only been trained for 1 epoch on a tiny dataset. The model hasn't learned coherent language patterns yet.

In [29]:
from transformers import pipeline

model_id = "damienbenveniste/mistral-pretraining"
pipe = pipeline("text-generation", model=model_id)
txt = "How are you?"
results = pipe(txt, num_return_sequences=1)
results[0]["generated_text"]


config.json:   0%|          | 0.00/628 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/338M [00:00<?, ?B/s]

Device set to use mps:0


'How are you? ( 1991918) 1911119815) was 1612471) was a was a 190) was a the 18000 to 19190) was the 1, M and a name of the 1) was the 27, 1961197. The 1918 – 38) and his firsts of the 1) in 147, and the 1, the first he was the 197, he was the first to 197991914, the 181) and the first to the two.\n\n\n\n\nE 18260, the 17) was a first, and the first to the first the 177, the first and his first the 196) was first to 1.\n\nE of the first the 1988689717, 20203.\n\n\n17769293, the first to the 18849'

The model generates mostly nonsensical text - this is expected since it's only been trained for 1 epoch on a tiny dataset. It hasn't learned coherent language patterns yet.

# Stage 2: Supervised Fine-tuning (SFT)

Now we'll take our pretrained model and teach it to follow instructions using the Alpaca dataset. This transforms the model from a general text generator into an instruction-following assistant.

In [4]:
dataset = load_dataset("tatsu-lab/alpaca", split="train[:1000]")

In [61]:
dataset['instruction'][0]

'Give three tips for staying healthy.'

In [62]:
dataset[0]['text']

'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'

The formatted text includes prompt structure with "### Instruction:" and "### Response:" markers. This teaches the model to understand and respond to instructions in this specific format.

In [63]:
dataset[0]

{'instruction': 'Give three tips for staying healthy.',
 'input': '',
 'output': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
 'text': 'Below is an instruction that describes a task. Write a response that appropriately completes the request.\n\n### Instruction:\nGive three tips for staying healthy.\n\n### Response:\n1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.'}

We load our pretrained model and tokenizer from the previous step.

In [5]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "damienbenveniste/mistral-pretraining"
model = AutoModelForCausalLM.from_pretrained(model_id)
tokenizer = AutoTokenizer.from_pretrained(model_id)

We can prepare the data for prompt completion by highlighting the prompt and the completion:

In [66]:
def transform(sample):
    prompt = sample['instruction']
    if sample['input']:
        prompt += f"\n\nInput: {sample['input']}"

    sample['prompt'] = prompt
    sample['completion'] = sample['output']

    return sample

dataset_completion = dataset.map(transform)

print(dataset_completion['prompt'][5])


Identify the odd one out.

Input: Twitter, Instagram, Telegram


In [68]:
print(dataset_completion['completion'][5])

Telegram


This code configures and runs supervised fine-tuning using TRL's SFTTrainer:

  **`SFTConfig`** - Configuration object that defines training parameters including output directory, dataset field containing text data, sequence length limits, training epochs, and
  hub settings.

  **`SFTTrainer`** - The main training class that handles supervised fine-tuning with the specified model, dataset, data collator (for completion-only training), and tokenizer as the
  processing class.

  **`trainer.train()`** - Executes the training loop with the configured parameters, fine-tuning the model on the instruction-response dataset while only computing loss on response
  tokens.

In [ ]:
from trl import SFTTrainer, SFTConfig

args = SFTConfig(
    output_dir="mistral-supervised",
    max_seq_length=512,
    num_train_epochs=1,
    push_to_hub=True,
    report_to="none", 
)

trainer = SFTTrainer(
    model,
    args=args,
    train_dataset=dataset_completion,
    processing_class=tokenizer,
)

trainer.train()

/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss


TrainOutput(global_step=125, training_loss=6.81018212890625, metrics={'train_runtime': 15.9571, 'train_samples_per_second': 62.668, 'train_steps_per_second': 7.834, 'total_flos': 62504624590848.0, 'train_loss': 6.81018212890625})

And we can push to the hub

In [59]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-supervised/commit/27accbbd44700297b7f82a965742aec060f7fa32', commit_message='End of training', commit_description='', oid='27accbbd44700297b7f82a965742aec060f7fa32', pr_url=None, repo_url=RepoUrl('https://huggingface.co/damienbenveniste/mistral-supervised', endpoint='https://huggingface.co', repo_type='model', repo_id='damienbenveniste/mistral-supervised'), pr_revision=None, pr_num=None)

## Train for chat 

In [6]:
def transform(sample):

    prompt = sample['instruction']
    if sample['input']:
        prompt += f"\n\nInput: {sample['input']}"

    messages = [
        {'role': 'user', 'content': prompt},
        {'role': 'assistant', 'content': sample['output']}

    ]

    sample['messages'] = messages

    return sample

dataset_chat = dataset.map(transform)

In [ ]:
from trl import clone_chat_template
# Set up the chat format
model, tokenizer = clone_chat_template(model, tokenizer, "Qwen/Qwen3-0.6B", )

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

The new embeddings will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`
The new lm_head weights will be initialized from a multivariate normal distribution that has old embeddings' mean and covariance. As described in this article: https://nlp.stanford.edu/~johnhew/vocab-expansion.html. To disable this, use `mean_resizing=False`


In [8]:
tokenizer.chat_template

'{%- if tools %}\n    {{- \'<|im_start|>system\\n\' }}\n    {%- if messages[0].role == \'system\' %}\n        {{- messages[0].content + \'\\n\\n\' }}\n    {%- endif %}\n    {{- "# Tools\\n\\nYou may call one or more functions to assist with the user query.\\n\\nYou are provided with function signatures within <tools></tools> XML tags:\\n<tools>" }}\n    {%- for tool in tools %}\n        {{- "\\n" }}\n        {{- tool | tojson }}\n    {%- endfor %}\n    {{- "\\n</tools>\\n\\nFor each function call, return a json object with function name and arguments within <tool_call></tool_call> XML tags:\\n<tool_call>\\n{\\"name\\": <function-name>, \\"arguments\\": <args-json-object>}\\n</tool_call><|im_end|>\\n" }}\n{%- else %}\n    {%- if messages[0].role == \'system\' %}\n        {{- \'<|im_start|>system\\n\' + messages[0].content + \'<|im_end|>\\n\' }}\n    {%- endif %}\n{%- endif %}\n{%- set ns = namespace(multi_step_tool=true, last_query_index=messages|length - 1) %}\n{%- for message in messa

In [9]:
dataset_chat['messages'][0]

[{'content': 'Give three tips for staying healthy.', 'role': 'user'},
 {'content': '1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. \n2. Exercise regularly to keep your body active and strong. \n3. Get enough sleep and maintain a consistent sleep schedule.',
  'role': 'assistant'}]

In [14]:
res = tokenizer.apply_chat_template(
    dataset_chat['messages'][0],
    tokenize=False, 
    add_generation_prompt=True, 
)

print(res)

<|im_start|>user
Give three tips for staying healthy.<|im_end|>
<|im_start|>assistant
<think>

</think>

1.Eat a balanced diet and make sure to include plenty of fruits and vegetables. 
2. Exercise regularly to keep your body active and strong. 
3. Get enough sleep and maintain a consistent sleep schedule.<|im_end|>
<|im_start|>assistant



In [16]:
from trl import SFTTrainer, SFTConfig

args = SFTConfig(
    output_dir="mistral-supervised-chat",
    max_seq_length=512,
    num_train_epochs=1,
    push_to_hub=True,
    report_to="none", 
    bf16=False
)

trainer = SFTTrainer(
    model,
    args=args,
    train_dataset=dataset_chat,
    processing_class=tokenizer,
)

trainer.train()

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


Tokenizing train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)


Step,Training Loss
10,7.130100
20,6.563200
30,6.687300
40,6.399900
50,6.269600
60,6.237800
70,6.090600
80,6.122300
90,6.205300
100,6.044600


TrainOutput(global_step=125, training_loss=6.320494308471679, metrics={'train_runtime': 35.332, 'train_samples_per_second': 28.303, 'train_steps_per_second': 3.538, 'total_flos': 68335202562048.0, 'train_loss': 6.320494308471679})

In [17]:
trainer.push_to_hub()

CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-supervised-chat/commit/88a450ff36bf33af8c41312bb77d1111806ad343', commit_message='End of training', commit_description='', oid='88a450ff36bf33af8c41312bb77d1111806ad343', pr_url=None, repo_url=RepoUrl('https://huggingface.co/damienbenveniste/mistral-supervised-chat', endpoint='https://huggingface.co', repo_type='model', repo_id='damienbenveniste/mistral-supervised-chat'), pr_revision=None, pr_num=None)

## RLHF Data Preparation

The HH-RLHF dataset contains pairs of responses: one chosen (preferred) and one rejected (less preferred) for the same conversation. This data captures human preferences about response quality.

In [21]:
dataset = load_dataset("Anthropic/hh-rlhf", split='train[:1000]')

This shows a "chosen" response that was preferred by human annotators. The dataset contains pairs of chosen vs rejected responses for the same conversation.

In [22]:
print(dataset['chosen'][0])



Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: I haven't even thought about it.


In [23]:
print(dataset['rejected'][0])



Human: What are some cuss words in english?

Assistant: Here’s an incomplete list.

Ass, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,

Human: What's your favorite one?

Assistant: Ass.


Let's now train a reward model

In [37]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_id = 'damienbenveniste/mistral-supervised'

reward_model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at damienbenveniste/mistral-supervised and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [27]:
reward_model

MistralForSequenceClassification(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 768, padding_idx=2)
    (layers): ModuleList(
      (0-3): 4 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear(in_features=768, out_features=768, bias=False)
          (k_proj): Linear(in_features=768, out_features=384, bias=False)
          (v_proj): Linear(in_features=768, out_features=384, bias=False)
          (o_proj): Linear(in_features=768, out_features=768, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear(in_features=768, out_features=3072, bias=False)
          (up_proj): Linear(in_features=768, out_features=3072, bias=False)
          (down_proj): Linear(in_features=3072, out_features=768, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((768,), eps=1e-06)
        (post_attention_layernorm): MistralRMSNorm((768,), eps=1e-06)
      )
    )
    (norm): MistralRMSNorm((768,),

In [28]:
from trl import RewardTrainer, RewardConfig

reward_config = RewardConfig(
    output_dir="mistral-reward",
    num_train_epochs=1,
    push_to_hub=True,
    report_to="none",
    bf16=False
)

trainer = RewardTrainer(
    model=reward_model,
    args=reward_config,
    train_dataset=dataset,
    processing_class=tokenizer
)

trainer.train()

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,0.740200
20,0.764700
30,0.700200
40,0.698500
50,0.674700
60,0.678000
70,0.627200
80,0.723500
90,0.693400
100,0.661700


TrainOutput(global_step=125, training_loss=0.6901308479309082, metrics={'train_runtime': 124.4076, 'train_samples_per_second': 8.038, 'train_steps_per_second': 1.005, 'total_flos': 0.0, 'train_loss': 0.6901308479309082, 'epoch': 1.0})

And we can push to the hub

In [29]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-reward/commit/60ea73c27b89864e12c35f9ebdc9a8f1b3d7f6f8', commit_message='End of training', commit_description='', oid='60ea73c27b89864e12c35f9ebdc9a8f1b3d7f6f8', pr_url=None, repo_url=RepoUrl('https://huggingface.co/damienbenveniste/mistral-reward', endpoint='https://huggingface.co', repo_type='model', repo_id='damienbenveniste/mistral-reward'), pr_revision=None, pr_num=None)

## Training with chat template

In [33]:
import re

def extract_messages(text):
      pattern = r'(Human|Assistant):\s*(.*?)(?=\n\n(?:Human|Assistant):|$)'
      matches = re.findall(pattern, text, re.DOTALL)

      messages = []
      for role, content in matches:
          if role == 'Human':
              messages.append({"role": "user", "content": content.strip()})
          elif role == 'Assistant':
              messages.append({"role": "assistant", "content": content.strip()})

      return messages

def transform(sample):

    sample['chosen'] = extract_messages(sample['chosen'])
    sample['rejected'] = extract_messages(sample['rejected'])

    return sample

dataset_chat = dataset.map(transform)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

In [34]:
dataset_chat['chosen'][0]

[{'content': 'What are some cuss words in english?', 'role': 'user'},
 {'content': 'Here’s an incomplete list.\n\nAss, dick, bugger, crap, fuck, shit, bitch, turd, shithead, shitbag, scrotum, cunt, whore, fucker, shit-eating, cum, cumbucket, fucknugget, butthole, poop, jackass, cocksucker, asshole, goddamn, piss, sperm, blow, wank, jism, cum-sucking, masturbate, faggot, queer, jizz, jizz-licking, prostitute, slut, cheater, fornicator, floozy, wetback, Mexican, Hispanic, sodomite, midget, mama’s boy, faggot, pervert, queer, scumbag, bitch,',
  'role': 'assistant'},
 {'content': "What's your favorite one?", 'role': 'user'},
 {'content': "I haven't even thought about it.", 'role': 'assistant'}]

In [43]:
from trl import clone_chat_template
from transformers import GenerationConfig                                                                                                

if reward_model.generation_config is None:                                                                                                      
    reward_model.generation_config = GenerationConfig.from_model_config(reward_model.config)                                                          
                                                                                                                                                        
reward_model, tokenizer = clone_chat_template(reward_model, tokenizer, "Qwen/Qwen3-0.6B", )                                                        


In [44]:
from trl import RewardTrainer, RewardConfig

reward_config = RewardConfig(
    output_dir="mistral-reward-chat",
    num_train_epochs=1,
    push_to_hub=True,
    report_to="none",
    bf16=False
)

trainer = RewardTrainer(
    model=reward_model,
    args=reward_config,
    train_dataset=dataset_chat,
    processing_class=tokenizer
)

trainer.train()

average_tokens_across_devices is set to True but it is invalid when world size is1. Turn it to False automatically.


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

Filter:   0%|          | 0/1000 [00:00<?, ? examples/s]

/Users/damienbenveniste/Projects/Teaching/The-AiEdge-Code-Examples/myenv/lib/python3.13/site-packages/torch/utils/data/dataloader.py:683: UserWarning: 'pin_memory' argument is set as true but not supported on MPS now, then device pinned memory won't be used.
  warnings.warn(warn_msg)
You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss
10,0.699700
20,0.713300
30,0.689100
40,0.652900
50,0.664000
60,0.680100
70,0.576700
80,0.820700
90,0.722000
100,0.639400


TrainOutput(global_step=125, training_loss=0.680673526763916, metrics={'train_runtime': 121.2032, 'train_samples_per_second': 8.251, 'train_steps_per_second': 1.031, 'total_flos': 0.0, 'train_loss': 0.680673526763916, 'epoch': 1.0})

In [45]:
trainer.push_to_hub()

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-reward-chat/commit/15105f0f06305c3c6069893a77b25e6dc35eecde', commit_message='End of training', commit_description='', oid='15105f0f06305c3c6069893a77b25e6dc35eecde', pr_url=None, repo_url=RepoUrl('https://huggingface.co/damienbenveniste/mistral-reward-chat', endpoint='https://huggingface.co', repo_type='model', repo_id='damienbenveniste/mistral-reward-chat'), pr_revision=None, pr_num=None)

## Step 3: PPO Training 

With our reward model trained, we can now use PPO to optimize our SFT model. The model will generate responses and receive reward scores, learning to produce higher-quality outputs over time. For PPO training, we load a different dataset - the last 1000 examples from Alpaca. We'll use these as prompts for the model to generate responses.

In [46]:
dataset = load_dataset("tatsu-lab/alpaca", split="train[-1000:]")

Let's see another example from the dataset to understand the format:

In [47]:
print(dataset['text'][1])

Below is an instruction that describes a task, paired with an input that provides further context. Write a response that appropriately completes the request.

### Instruction:
Given a story, (add/edit/compare/remove) an element from it.

### Input:
Once upon a time there was a little girl who loved to read books.

### Response:
Once upon a time there was a little girl who loved to read books and play with her pet rabbit.


  This code loads the necessary components for PPO (Proximal Policy Optimization) training from a pre-trained supervised fine-tuned model:

  **`model_id`** - Specifies the Hugging Face model identifier for the previously fine-tuned Mistral model that will serve as the base for PPO training.

  **`ppo_model`** - Loads the causal language model that will be optimized during PPO training to generate responses aligned with human preferences.

  **`value_model`** - Loads a sequence classification model with a single output (scalar value) that estimates the value of generated sequences for the PPO algorithm.

  **`tokenizer`** - Loads the tokenizer associated with the model to handle text preprocessing and encoding for both models during training.

In [48]:
model_id = 'damienbenveniste/mistral-supervised'

ppo_model = AutoModelForCausalLM.from_pretrained(model_id)
value_model = AutoModelForSequenceClassification.from_pretrained(model_id, num_labels=1)
tokenizer = AutoTokenizer.from_pretrained(model_id)

Some weights of MistralForSequenceClassification were not initialized from the model checkpoint at damienbenveniste/mistral-supervised and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [54]:
from trl import clone_chat_template
from transformers import GenerationConfig                                                                                                
                                                                                                                                                                                                           
ppo_model, tokenizer = clone_chat_template(ppo_model, tokenizer, "Qwen/Qwen3-0.6B", )  

Each text contains the full instruction-response example. For PPO, we'll extract just the instruction part as prompts.


 This code prepares the dataset for PPO training by extracting prompts and tokenizing them:

  **`tokenize()`** - Function that splits each sample at the "### Response" delimiter to extract only the instruction/prompt portion, then tokenizes it and stores both the token IDs
  and raw prompt text.

  **`tokenized_dataset`** - Applies the tokenization function to each sample in the dataset, processing them individually rather than in batches.

  **`set_format()`** - Converts the dataset to PyTorch tensor format, making it compatible with the PPO training pipeline that expects tensor inputs.

In [55]:
def tokenize_completion(sample):
    prompt = sample['instruction']
    if sample['input']:
        prompt += f"\n\nInput: {sample['input']}"

    sample["input_ids"] = tokenizer.encode(prompt)
    return sample

def tokenize_chat(sample):
    prompt = sample['instruction']
    if sample['input']:
        prompt += f"\n\nInput: {sample['input']}"

    messages = [
        {'role': 'user', 'content': prompt},
    ]

    sample["input_ids"] = tokenizer.apply_chat_template(messages)
    return sample

tokenized_dataset = dataset.map(tokenize_chat, batched=False, remove_columns=dataset.column_names)
tokenized_dataset.set_format(type="torch")


Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

This code sets up and runs PPO (Proximal Policy Optimization) training to align the language model with human preferences:

**`PPOConfig`** - Configuration object that specifies PPO training parameters including output directory and batch sizes for gradient updates.

**`PPOTrainer`** - The main PPO training class that orchestrates the reinforcement learning process using the policy model, reward model, value model, and datasets.

**`ppo_trainer.train()`** - Executes the PPO training loop where the model generates responses, receives rewards from the reward model, and updates its policy to maximize human 
preference alignment.

In [56]:
from trl import PPOConfig, PPOTrainer

ppo_config = PPOConfig(
    output_dir="mistral-ppo",
    mini_batch_size=2,
    batch_size=2,
    bf16=False
)

ppo_trainer = PPOTrainer(
    model=ppo_model,
    args=ppo_config,
    train_dataset=tokenized_dataset,
    eval_dataset=tokenized_dataset,
    processing_class=tokenizer,
    reward_model=reward_model,
    value_model=value_model,  
    ref_model=None 
)

ppo_trainer.train()

`generation_config` default values have been modified to match model-specific defaults: {'bos_token_id': 1}. If this is not desired, please set these values explicitly.


===training policy===


Step,Training Loss


In [57]:
ppo_trainer.push_to_hub()

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

model.safetensors:   0%|          | 0.00/339M [00:00<?, ?B/s]

training_args.bin:   0%|          | 0.00/6.54k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/damienbenveniste/mistral-ppo/commit/39ae472b3a41bf181439aa3d45c39af5d529b7de', commit_message='End of training', commit_description='', oid='39ae472b3a41bf181439aa3d45c39af5d529b7de', pr_url=None, repo_url=RepoUrl('https://huggingface.co/damienbenveniste/mistral-ppo', endpoint='https://huggingface.co', repo_type='model', repo_id='damienbenveniste/mistral-ppo'), pr_revision=None, pr_num=None)